## AWS Data Lake Demo

- Template Only (Not TEsted or Run)

In [1]:
import boto3, botocore
from botocore.exceptions import ClientError
import os, time, json, io, zipfile
from datetime import date
from dotenv import load_dotenv


from misc import load_from_yaml, save_to_yaml
import iam, s3, lf, rds, vpc, ec2

load_dotenv(".env")
# boto3.setup_default_session(profile_name="AMominNJ")

In [ ]:
ACCOUNT_ID        = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION            = os.environ['AWS_DEFAULT_REGION']
VPC_ID            = os.environ['AWS_DEFAULT_VPC']
SECURITY_GROUP_ID = os.environ['AWS_DEFAULT_SG_ID']
SUBNET_IDS        = SUBNET_IDS = os.environ["AWS_DEFAULT_SUBNET_IDS"].split(":")
SUBNET_ID         = SUBNET_IDS[0]
print(SUBNET_IDS)

In [2]:
sts_client           = boto3.client('sts')
rds_client           = boto3.client('rds')
iam_client           = boto3.client('iam')
s3_client            = boto3.client('s3')
glue_client          = boto3.client('glue')
lakeformation_client = boto3.client('lakeformation')
ec2_client           = boto3.client('ec2', region_name='us-east-1')
ec2_resource         = boto3.resource('ec2', region_name='us-east-1')

#### Create IAM Role (for AWS Glue Service)

- Create aws glue role by the name of `glue_role_name`.
- Assign Power User Access Policy (`PowerUserAccess`) to the role.

In [7]:
GLUE_ROLE_NAME = 'glue-rds-role_name'

In [8]:
assume_role_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "glue.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
GLUE_ROLE_ARN = iam_client.create_role(
    RoleName=GLUE_ROLE_NAME,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
    Description="Glue Service Role"
)['Role']['Arn']

In [9]:
aws_glue_service_policy_arn = "arn:aws:iam::aws:policy/service-role/AWSGlueServiceRole"
# admin_access_policy_arn = "arn:aws:iam::aws:policy/AdministratorAccess"
# power_user_access_policy_arn = "arn:aws:iam::aws:policy/PowerUserAccess"

In [ ]:
# Attach AWS managed policy with the role
iam_client.attach_role_policy(
    RoleName=GLUE_ROLE_NAME,
    PolicyArn=aws_glue_service_policy_arn
)

#### Create S3 Bucket and Folders

In [11]:
S3_BUCKET_DATA_LAKE = "httx-datalake-bkt"
S3_BUCKET_GLUE_ASSETS = "httx-glue-assets-bkt"

In [ ]:
acl = 'private'                             # Set the ACL (e.g., 'private', 'public-read')
enable_versioning = False                   # Enable versioning
enable_encryption = False                   # Enable server-side encryption

folders = ['temporary', 'sparkHistoryLogs']

s3.create_s3_bucket(S3_BUCKET_DATA_LAKE, REGION, enable_versioning, enable_encryption)
s3.create_s3_bucket(S3_BUCKET_GLUE_ASSETS, REGION, enable_versioning, enable_encryption, folders)

#### Create Glue Catalog Database

In [13]:
CATALOG_DB_NAME = 'httx-catalog-db'

In [ ]:
## Example usage
DATALAKE_LOCATION_URI = f"s3://{S3_BUCKET_DATA_LAKE}"

create_database_response = glue_client.create_database(
    CatalogId=ACCOUNT_ID,
    DatabaseInput={
        'Name': CATALOG_DB_NAME,
        'Description': 'TICKIT: A platform that brings together buyers and sellers of tickets to entertainment events',
        'LocationUri': DATALAKE_LOCATION_URI,
    }
)
print(create_database_response)

In [16]:
# glue_client.update_database(
#     CatalogId=ACCOUNT_ID,
#     Name=CATALOG_DB_NAME,
#     DatabaseInput={
#         'Name': CATALOG_DB_NAME,
#         'UseOnlyIamAccessControl': False
#     }
# )
# lf.register_s3_path_as_data_lake_location(LFDB_LOCATION_URI)

#### Create RDS Databases & it's Resources

In [17]:
DB_NAME='glue-rds-db'
DB_USERNAME = os.environ['USERNAME']
DB_PASSWORD = os.environ['PASSWORD']
JDBC_PASSWORD = os.environ['PASSWORD'] # Password for RDS Databases

In [ ]:
SUBNET_GROUP_NAME = 'httx-rds-subnet-group'
## Create the RDS subnet group
response = rds_client.create_db_subnet_group(
    DBSubnetGroupName=SUBNET_GROUP_NAME,
    DBSubnetGroupDescription='Subnet group for RDS instance',
    SubnetIds=SUBNET_IDS
)
print(response)

In [21]:
instances = [
    {
        'db_instance_identifier': 'httx-rds-mysql',
        'db_name': 'httx_mysql',
        'db_username': DB_USERNAME,
        'db_password': JDBC_PASSWORD,
        'engine': 'mysql',
        'port': 3306,
        'engine_version': '8.0.32',
        'db_instance_class': 'db.t3.micro',
        'allocated_storage': 20,
        'availability_zone': 'us-east-1a',
        'tags': [{'Key': 'Project', 'Value': 'glue-rds-Crawler'}],
        'security_group_ids': [SECURITY_GROUP_ID],
        'db_subnet_group_name': SUBNET_GROUP_NAME,
    },
    {
        'db_instance_identifier': 'httx-rds-mssql',
        'db_name': '',
        'db_username': DB_USERNAME,
        'db_password': JDBC_PASSWORD,
        'port': 1433,
        'engine': 'sqlserver-ex',
        'engine_version': '15.00.4153.1.v1',
        'db_instance_class': 'db.t3.micro',
        'allocated_storage': 20,
        'availability_zone': 'us-east-1a',
        'tags': [{'Key': 'Project', 'Value': 'glue-rds-Crawler'}],
        'security_group_ids': [SECURITY_GROUP_ID],
        'db_subnet_group_name': SUBNET_GROUP_NAME,
    },
    {
        'db_instance_identifier': 'httx-rds-postgresql',
        'db_name': 'httx_postgresql',
        'db_username': DB_USERNAME,
        'db_password': JDBC_PASSWORD,
        'port': 5432,
        'engine': 'postgres',
        'engine_version': '14.13',
        'db_instance_class': 'db.t3.micro',
        'allocated_storage': 20,
        'availability_zone': 'us-east-1a',
        'tags': [{'Key': 'Project', 'Value': 'glue-rds-Crawler'}],
        'security_group_ids': [SECURITY_GROUP_ID],
        'db_subnet_group_name': SUBNET_GROUP_NAME,
    }
]

In [ ]:
rds.create_rds_instance(**instances[0])   # 'httx-rds-mysql'
rds.create_rds_instance(**instances[1])   # 'httx-rds-mssql'
rds.create_rds_instance(**instances[2])   # 'httx-rds-postgresql'

-   `Gateway` endpoints serve as a target for a route in your route table for traffic destined for the service.

In [23]:
# VPC Endpoint parameters
VPC_ENDPOINT_TAG = 'rds-glue-vpc-endpoint'
VPC_ENDPOINT_SERVICE_NAME = 'com.amazonaws.us-east-1.s3'
SECURITY_GROUP_IDS = [SECURITY_GROUP_ID]  # Security group(s) associated with the endpoint
ROUTE_TABLE_IDS = ['rtb-0ec4311296ec952f8']

# Create an Interface Endpoint
VPC_ENDPOINT_ID = ec2_client.create_vpc_endpoint(
    VpcEndpointType='Gateway',
    VpcId=VPC_ID,
    ServiceName=VPC_ENDPOINT_SERVICE_NAME,
    RouteTableIds=ROUTE_TABLE_IDS,
    # SubnetIds=sg_id,
    # SecurityGroupIds=security_group_ids,
    PrivateDnsEnabled=False  # Enable private DNS to resolve service names within the VPC
)['VpcEndpoint']['VpcEndpointId']

In [ ]:
ec2_client.create_tags(Resources=[VPC_ENDPOINT_ID],Tags=[{'Key': 'Name', 'Value': 'rds_vpc_endpoint'}])

##### Load sql data from Local Machine to RDS Instance

-   Load into Postgresql (TESTED):

    -   `$ brew install postgresql` -> Install PostgreSQL Client
    -   `$ psql -h <rds-endpoint> -p <port> -U <username> -d <dbname>` -> Connect via Command Line if needed
    -   `$ PGPASSWORD={DB_PASSWORD} psql -h {postgresql_endpoint} -p {rds_instances[0]['port']} -U {DB_USERNAME} -d interview_questions < /Users/am/mydocs/Software_Development/Web_Development/aws/aws_rds/interview_questions.sql`

-   Load into MySQL (TESTED):

    -   `$ brew install postgresql` -> Install PostgreSQL Client
    -   `$ mysql -h <rds-endpoint> -p <port> -U <username> -d <dbname>` -> Connect via Command Line if needed
    -   `$ mysql -h {mysql_endpoint} -P {mysql_port} -u httxadmin -p'{DB_PASSWORD}' interview_questions < /Users/am/mydocs/Software_Development/Web_Development/aws/aws_rds/interview_questions.sql`

-   Load into MSSQL:

    -   Install `sqlcmd` Client on the local machine
    -   `$ sqlcmd -S <rds_mssql_endpoint>,<mssql_port> -U <httxadmin> -P <DB_PASSWORD> -d <dbname>` -> Connect via Command Line if needed
    -   `$ sqlcmd -S {mssql_endpoint},{mssql_por}> -U {httxadmin} -P {DB_PASSWORD} -d {database_name} -i /Users/am/mydocs/Software_Development/Web_Development/aws/aws_rds/interview_questions.sql`


#### Create Glue Resources

In [ ]:
MSSQL_CONNECTION_NAME = 'tickit_mssql_connection'
MYSQL_CONNECTION_NAME = 'tickit_mysql_connection'
POSTGRESQL_CONNECTION_NAME = 'tickit_postgresql_connection'

MSSQL_CRAWLER_NAME = "httx-mssqlcrawler"
MYSQL_CRAWLER_NAME = "httx-mysqlcrawler"
POSTGRESQL_CRAWLER_NAME = "httx-postgresqlcrawler"

In [ ]:
mssql_endpoint = rds.get_rds_endpoint(rds.get_rds_endpoint(instances[0]['db_instance_identifier']))
mysql_endpoint = rds.get_rds_endpoint(rds.get_rds_endpoint(instances[1]['db_instance_identifier']))
postgresql_endpoint = rds.get_rds_endpoint(rds.get_rds_endpoint(instances[2]['db_instance_identifier']))

In [ ]:
mysql_connection_url = f"jdbc:sqlserver://{rds.get_rds_endpoint(instances[0]['db_instance_identifier'])}:{instances[0]['port']}/{instances[0]['db_name']}"
mssql_connection_url = f"jdbc:sqlserver://{rds.get_rds_endpoint(instances[1]['db_instance_identifier'])}:{instances[1]['port']}/{instances[1]['db_name']}"
postgresql_connection_url = f"jdbc:sqlserver://{rds.get_rds_endpoint(instances[2]['db_instance_identifier'])}:{instances[2]['port']}/{instances[2]['db_name']}"

glue.create_glue_connection(MYSQL_CONNECTION_NAME, mysql_connection_url, DB_USERNAME, DB_PASSWORD, SECURITY_GROUP_ID, SUBNET_ID, REGION)
glue.create_glue_connection(MSSQL_CONNECTION_NAME, mssql_connection_url, DB_USERNAME, DB_PASSWORD, SECURITY_GROUP_ID, SUBNET_ID, REGION)
glue.create_glue_connection(POSTGRESQL_CONNECTION_NAME, postgresql_connection_url, DB_USERNAME, DB_PASSWORD, SECURITY_GROUP_ID, SUBNET_ID, REGION)

In [ ]:
glue.create_glue_jdbc_crawler(MYSQL_CRAWLER_NAME, MYSQL_CONNECTION_NAME, GLUE_ROLE_ARN, CATALOG_DB_NAME, 'ecomm/%')
glue.create_glue_jdbc_crawler(MSSQL_CRAWLER_NAME, MSSQL_CONNECTION_NAME, GLUE_ROLE_ARN, CATALOG_DB_NAME, 'tickit/crm/%')
glue.create_glue_jdbc_crawler(POSTGRESQL_CRAWLER_NAME, POSTGRESQL_CONNECTION_NAME, GLUE_ROLE_ARN, CATALOG_DB_NAME, 'tickit/ems/%')

In [ ]:

# Step 2: Create Glue Jobs (Bronze, Silver, and so on)
glue_jobs = [
    {
        'name': 'tickit_silver_ems_venue',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/silver/tickit_silver_ems_venue.py"
    },
    {
        'name': 'tickit_silver_ems_event',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/silver/tickit_silver_ems_event.py"
    },
    {
        'name': 'tickit_silver_ems_category',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/silver/tickit_silver_ems_category.py"
    },
    {
        'name': 'tickit_silver_crm_user',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/silver/tickit_silver_crm_user.py"
    },
    {
        'name': 'tickit_silver_ecomm_sale',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/silver/tickit_silver_ecomm_sale.py"
    },
    {
        'name': 'tickit_silver_ecomm_listing',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/silver/tickit_silver_ecomm_listing.py"
    },
    {
        'name': 'tickit_silver_ecomm_date',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/silver/tickit_silver_ecomm_date.py"
    },
    {
        'name': 'tickit_bronze_ems_category',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/bronze/tickit_bronze_ems_category.py"
    },
    {
        'name': 'tickit_bronze_ems_event',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/bronze/tickit_bronze_ems_event.py"
    },
    {
        'name': 'tickit_bronze_ems_venue',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/bronze/tickit_bronze_ems_venue.py"
    },
    {
        'name': 'tickit_bronze_crm_user',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/bronze/tickit_bronze_crm_user.py"
    },
    {
        'name': 'tickit_bronze_ecomm_date',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/bronze/tickit_bronze_ecomm_date.py"
    },
    {
        'name': 'tickit_bronze_ecomm_listing',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/bronze/tickit_bronze_ecomm_listing.py"
    },
    {
        'name': 'tickit_bronze_ecomm_sale',
        'script': f"s3://{S3_BUCKET_GLUE_ASSETS}/glue_job_scripts/bronze/tickit_bronze_ecomm_sale.py"
    },
]

TEM_DIR = f"s3://{S3_BUCKET_GLUE_ASSETS}/temporary/"
SPARK_EVENT_LOG_PATH = f"s3://{S3_BUCKET_GLUE_ASSETS}/sparkHistoryLogs/"

for job in glue_jobs: 
    glue.create_glue_job(job['name'], job['script'], GLUE_ROLE_ARN, TEM_DIR, SPARK_EVENT_LOG_PATH)

#### Delete Resources

In [ ]:
# lakeformation_client.deregister_resource(ResourceArn=f'arn:aws:s3:::{bucket_name}')
# glue_client.delete_database(CatalogId=catalog_id,Name=catalog_db_name)

In [ ]:
# s3 = boto3.resource('s3')
# bucket = s3.Bucket(bucket_name)

# # Delete all objects in the bucket
# bucket.objects.all().delete()

# # Delete all object versions (if versioning is enabled)
# # bucket.object_versions.all().delete()

# # Finally, delete the bucket
# bucket.delete()


In [ ]:
# rds_client.delete_db_subnet_group(DBSubnetGroupName=SUBNET_GROUP_NAME)

In [ ]:
# response = rds_client.delete_db_subnet_group(DBSubnetGroupName=SUBNET_GROUP_NAME)

In [26]:
# rds.delete_rds_instance(instances[0]['db_instance_identifier'])
# rds.delete_rds_instance(instances[1]['db_instance_identifier'])
# rds.delete_rds_instance(instances[2]['db_instance_identifier'])

In [ ]:
## Delete the VPC Endpoint
# response = ec2_client.delete_vpc_endpoints(VpcEndpointIds=[vpc_endpoint_id])

In [ ]:
# response = glue_client.delete_connection(ConnectionName=glue_mysql_connection_name)
# response = glue_client.delete_crawler(Name=rds_crawler_name)

In [ ]:
# for job in glue_jobs: glue_client.delete_job(JobName=job['name'])

In [ ]:
# DELETE IAM ROLE AT THE END AFTER DELETING ALL OTHER RESOURCES.
# iam.delete_iam_role(GLUE_ROLE_NAME)